In [ ]:
pip install gemact

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.1 MB/s 
  Attempting uninstall: gemact
    Found existing installation: gemact 0.1.3
    Uninstalling gemact-0.1.3:
      Successfully uninstalled gemact-0.1.3


In [ ]:
import gemact
import numpy as np

In [ ]:
gemact.__version__

'0.1.4'

# Different models to compute the aggregate cost

Within the framework of the collective risk model \cite[p.~148]{klugman98} the aggregate loss $X$ is modeled as

$$
X=\sum_{i=1}^{N} Z_i,
$$

where  $N: \Omega \rightarrow \mathbb{N}_0$ is the random variable modelling the number of claims and $Z_i : \Omega \rightarrow \mathbb{R^+},\;i=1,2,...,N$ is the payment amount for the $i$-th individual loss. The independence assumptions are:

1. Conditional on $N=n$, the random variables $Z_1,...,Z_n$ are i.i.d. and their distribution do not depend on $n$.
2. $N$ is independent of $Z_1, ..., ,Z_n$.


GEMAct requires distributive assumptions on frequency and severity. In the following tutorial we will assume:

* $N \sim Poisson(4)$
* $Z_i \sim Genpareto(shape=.2, scale=1, loc=0)$



In [ ]:
#poisson -genpareto

# define Poisson frequency model
frequency = Frequency(
    dist='poisson',
    par={'mu': 4}
    )

# define GenPareto severity model
severity = Severity(
    dist='genpareto',
    par={'c': .2, 'scale': 1}
    )

# define PolicyStructure: default is a empty policy, namely infinty-xs-0 XL with no Quota Share.
policystructure = PolicyStructure()

# create LossModel with above defined objects.
# aggr_loss_dist_method approach set to 'mc', i.e. Monte Carlo.
lm_mc = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure,
    aggr_loss_dist_method='mc',
    n_sim=1e+05,
    random_state=1
    )

INFO:lossmodel|..Approximating aggregate loss distribution via Monte Carlo simulation..
INFO:lossmodel|..Simulation completed..


In [ ]:
# create LossModel with above defined objects.
# aggr_loss_dist_method approach is here set to 'recursion', i.e. Panjer Recursion.
lm_rec = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure,
    aggr_loss_dist_method='recursion',
    n_sev_discr_nodes=int(1000),
    sev_discr_step=1,
    n_aggr_dist_nodes=int(10000)
    )

INFO:lossmodel|..Approximating aggregate loss distribution via Panjer recursion..
INFO:lossmodel|..Panjer recursion completed..


In [ ]:
# create LossModel with above defined objects.
# finally, aggr_loss_dist_method approach is set to 'fft', i.e. Fast Fourier Transform.
lm_fft = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure,
    aggr_loss_dist_method='fft',
    n_sev_discr_nodes=int(1000),
    sev_discr_step=1,
    n_aggr_dist_nodes=int(10000)
    )

INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [ ]:
# print averages of the aggregate loss distribution for the three LossModels.
print('MC: ',lm_mc.aggr_loss_mean())
print('RECURSIVE: ',lm_rec.aggr_loss_mean())
print('FFT: ',lm_fft.aggr_loss_mean())

MC 4.986130567647585
RECURSIVE 4.999999984652499
FFT 4.9999999846531935


# (Re)insurance contracts pricing

Below we show the following pricing models:

* Excess-of-loss (XL),
* Excess-of-loss with aggregate conditions (reinstatements),
* Stop-loss.

In [ ]:
# severity, frequency and policystructure can be created on-the-fly within LossModel definition.
# PolicyStructure has a 100-xs-100 XL Layer.
lmXL = LossModel(
    frequency=Frequency(
        dist='poisson',
        par={'mu': .5}
    ),
    severity=Severity(
        par= {'loc': 0,
        'scale': 83.33333333333334,
        'c': 0.8333333333333334},
        dist='genpareto'
    ),
    policystructure=PolicyStructure(
        layers=Layer(
            cover=100,
            deductible=100
        )
    ),
    aggr_loss_dist_method='fft',
    sev_discr_method='massdispersal',
    n_sev_discr_nodes=int(10000),
    sev_discr_step=.01,
    n_aggr_dist_nodes=int(100000)
    )

INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [ ]:
lmXL.pricing()

                     Contract specification            parameter                value
                               Deductible                    d                100.0
                                    Cover                u - d                100.0
                           Upper priority                    u                200.0
                     Aggregate deductible                    L                100.0
                     Quota share ceded portion                alpha                    1
                             Pure premium                    P   1.3354758296850144

 Reinstatement layer loading c:  []
fft 	 n_sev_discr_nodes m:  9999 	 n_aggr_dist_nodes n:  100000


# Excess-of-loss with aggregate conditions (reinstatements)


In [ ]:
# PolicyStructure has a 100-xs-0 XL Layer, 0 aggregate deductible, 1 reinstatement and 50% reinstatement loading.
# aggr_loss_dist_method is 'fft', namely Fast Fourier Transformation.
# sev_discr_method is 'massdispersal', namely mass dispersal method.
lmXLaggr = LossModel(
    frequency=Frequency(
        dist='poisson',
        par={'mu': .5}
    ),
    severity=Severity(
        par= {'loc': 0,
        'scale': 83.33333333333334,
        'c': 0.8333333333333334},
        dist='genpareto'
    ),
    policystructure=PolicyStructure(
        layers=Layer(
            cover=100,
            deductible=0,
            aggr_deductible=0,
            reinst_loading=0.5,
            n_reinst=1
        )
    ),
    aggr_loss_dist_method='fft',
    sev_discr_method='massdispersal',
    n_sev_discr_nodes=int(10000),
    sev_discr_step=.01,
    n_aggr_dist_nodes=int(100000)
    )


INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [ ]:
lmXLaggr.pricing()

                     Contract specification            parameter                value
                               Deductible                    d                  0.0
                                    Cover                u - d                100.0
                           Upper priority                    u                100.0
                     Aggregate deductible                    L                  0.0
                     Quota share ceded portion                alpha                    1
                     Number of reinstatements                    K                    1
                             Pure premium                    P    24.97739773475006

 Reinstatement layer loading c:  [1]
fft 	 n_sev_discr_nodes m:  9999 	 n_aggr_dist_nodes n:  100000


# Severity discretization

In order to pass from a continuous distribution to an arithmetic distribution, it is important to preserve the distribution properties either locally or globally. Given a span $h$ and $m$ the last available point, in GEMAct there are two available solutions.

**Method of mass dispersal**

$$ f_{0}=\operatorname{Pr}\left(Z<\frac{h}{2}\right)=F_{Z}\left(\frac{h}{2}-0\right)$$

$$ f_{j}=F_{Z}\left(j h+\frac{h}{2}-0\right)-F_{Z}\left(j h-\frac{h}{2}-0\right), \quad j=1,2, \ldots, m-1$$


**Method of local moments matching**

The following approach is applied to preserve the global mean of the distribution.

$$f_0 = m^0_0$$

$$f_j = m^{j}_0+ m^{j-1}_1 , \quad j=0,1, \ldots, m-1$$

$$ \sum_{j=0}^{1}\left(z_k+j h\right)^{r} m_{j}^{k}=\int_{z_k-0}^{z_k+ h-0} z^r d F_{Z}(z), \quad r=0,1$$

$$ m_{j}^{k}=\int_{z_k-0}^{z_k+p h-0} \prod_{i \neq j} \frac{x-z_k-i h}{(j-i) h} d F_{Z}(z), \quad j=0,1$$


Below we show the computations consistency by comparing the discrete mean via mass dispersal and the discrete mean via local moments discretization.

In [ ]:
# severity objects are equipped with a method for performing the discretization.
# this can be carried out using either 'massdispersal' or 'localmoments' argument.
massdispersal = severity.discretize(
    discr_method='massdispersal',
    n_discr_nodes=50000,
    discr_step=.01,
    deductible=0,
    cover=100
)

localmoments = severity.discretize(
    discr_method='localmoments',
    n_discr_nodes=50000,
    discr_step=.01,
    deductible=0,
    cover=100
)

meanMD = np.sum(massdispersal['sev_nodes'] * massdispersal['fj'])
meanLM = np.sum(localmoments['sev_nodes'] * localmoments['fj'])
print('Mean (mass dispersal): ', meanMD)
print('Mean (local moments): ', meanLM)

INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..
INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..
